In [49]:
# Run generator.js
import subprocess

params = ["200", "200"] # Enter the number of data. each prameter indicate the number of small, large map.

subprocess.run(["node", "generator.js"] + params)

CompletedProcess(args=['node', 'generator.js', '200', '200'], returncode=0)

In [50]:
# Run placer.py
import placer

maps = placer.load_maps()
for map in maps:
    placer.assign_parameters(map, enemy_density=0.05, cohesion=0.3, reward_density=0.01, range_multiplier=2, boss=True)
    placer.modify_map(map, group_min_dist=10, flag_try_count=50, enemy_sparsity=3)
placer.save_maps(maps)

In [51]:
# Run labeler.py
import labeler

labeler.label(file_num=1, difficulty_curve_interval=5)

In [52]:
# Create json file from base prompt
import json_creator
from config import base_prompt_path, json_base_prompt_path

json_creator.prompt_data_to_json(base_prompt_path, json_base_prompt_path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/4. prompt/base_prompt/AutoCOT1.txt'

In [ ]:
# Generate prompt
subprocess.run(["python", "prompt_generator.py"])

In [ ]:
# Preprocess
subprocess.run(["python", "preprocesser.py"])

In [58]:
# Run comparator.py
import comparator

comparator.compare(file_num=1)

In [59]:
# Get compared file
import os
import json

path = os.path.join(os.path.dirname(os.getcwd()), "data", "6. compared")

filename_list = [
    filename
    for filename in os.listdir(path)
    if filename.endswith(".json") and os.path.isfile(os.path.join(path, filename))
]

path_list = [os.path.join(path, filename) for filename in filename_list]

compared_list = list()
for file_path in path_list:
    with open(file_path, "r") as file:
        compared_batch = json.load(file)
        for compared in compared_batch["map_list"]:
            compared_list.append(compared)

In [60]:
# Calculate difference
diff_dict = dict()

params = [
    "map_size",
    "density",
    "empty_ratio",
    "exploration_requirement",
    "difficulty_curve",
    "nonlinearity"
]

for param in params:
    diff_dict[param] = list()

for compared in compared_list:
    for param in params:
        try:
            diff_dict[param].append([abs(float(compared["after_params"][param][i]) - float(compared["before_params"][param][i])) for i in range(len(compared["after_params"][param]))])
        except:
            diff_dict[param].append(abs(float(compared["after_params"][param]) - float(compared["before_params"][param])))

In [64]:
# Calculate mean and standard deviation
import numpy as np

mean_dict = dict()
std_dict = dict()

for param in params:
    try:
        mean_dict[param] = np.mean(np.array(diff_dict[param]), axis=0)
        std_dict[param] = np.std(np.array(diff_dict[param]), axis=0)
    except:
        mean_dict[param] = np.mean(diff_dict[param])
        std_dict[param] = np.std(diff_dict[param])

In [65]:
# Print mean and standard deviation
print(mean_dict)
print(std_dict)

{'map_size': array([ 6., 12.]), 'density': 0.006013016411997734, 'empty_ratio': 0.0388605923410677, 'exploration_requirement': 239.0, 'difficulty_curve': 0.16666666666666666, 'nonlinearity': 0.03187762792432691}
{'map_size': array([0., 0.]), 'density': 0.0, 'empty_ratio': 0.0, 'exploration_requirement': 0.0, 'difficulty_curve': 0.0, 'nonlinearity': 0.0}
